## Подготовка датасета

### Сегментация аудиодорожки

In [13]:
import sys
import os
import subprocess
import json
import math

import vosk
import librosa
import numpy as np
import pandas as pd

In [2]:
def extract_words(res):
    jres = json.loads(res)
    if not 'result' in jres:
        return []
    words = jres['result']
    return words

In [3]:
def transcribe_words(recognizer, bytes):
    result = []
    
    chunk_size = 4000
    for chunk_no in range(math.ceil(len(bytes) / chunk_size)):
        start = chunk_no * chunk_size
        end = min(len(bytes), (chunk_no + 1) * chunk_size)
        data = bytes[start : end]
        
        if recognizer.AcceptWaveform(data):
            words = extract_words(recognizer.Result())
            result += words
    result += extract_words(recognizer.FinalResult())
    
    return result

In [7]:
def get_info(input_path, out_path, model_path='vosk-model-small-ru-0.4'):
    vosk.SetLogLevel(-1)
    
    sample_rate = 16000
    audio, sr = librosa.load(input_path, sr=sample_rate)
    
    int16 = np.int16(audio * 32768).tobytes()
    
    model = vosk.Model(model_path)
    recognizer = vosk.KaldiRecognizer(model, sample_rate)
    
    res = transcribe_words(recognizer, int16)
    df = pd.DataFrame.from_records(res)
    df = df.sort_values('start')
    
    df.to_csv(out_path, index=False)
    return df

In [8]:
df = get_info('../audio.wav', 'out1.csv')

In [10]:
df.loc[df['conf'] == 1]

,conf,end,start,word
3,1.0,1.200000,0.904665,привет
4,1.0,1.410000,1.200000,ещё
5,1.0,1.590000,1.410000,раз
6,1.0,2.430000,1.710000,расскажу
11,1.0,4.501097,3.960000,началось
...,...,...,...,...
466,1.0,227.610000,227.520000,с
467,1.0,228.554619,227.610000,алгоритмами
468,1.0,229.320000,228.600000,плюсами
469,1.0,229.500000,229.320000,и


### Нарезка видео по получившимся таймингам

In [54]:
import moviepy.editor as ed
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [12]:
df_conf = df.loc[df['conf'] == 1]

In [14]:
word_dict = {}

In [26]:
for index, row in df_conf.iterrows():
    word_dict[row['word']] = []

In [27]:
for index, row in df_conf.iterrows():
    word_dict[row['word']].append((row['start'], row['end']))

In [55]:
for i in range(ord('а'), ord('я') + 1):
    os.mkdir(f'Dataset/{chr(i)}')

In [56]:
for key in word_dict:
    os.mkdir(f'Dataset/{key[0]}/{key}')

In [45]:
video_path = '../video1.mp4'
clip = ed.VideoFileClip(video_path)

In [68]:
for key in word_dict:
    for index, word in enumerate(word_dict[key]):
        ffmpeg_extract_subclip(video_path, word[0], word[1], 
                               f'Dataset/{key[0]}/{key}/{index}.mp4')

                                                                



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A





Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)


t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:06<00:39, 144.76it/s, now=None]



chunk: 100%|██████████| 1/1 [05:46<00:00, 346.75s/it, now=None]

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)


                                                                



t:   5%|▌         | 327/6023 [12:07<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:07<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:07<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:07<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:07<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:07<00:39, 144.76it/s, now=None]



chunk: 100%|██████████| 1/1 [05:47<00:00, 347.03s/it, now=None]

...

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)


                                                                



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



chunk: 100%|██████████| 1/1 [06:13<00:00, 373.55s/it, now=None]

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)


                                                                



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:33<00:39, 144.76it/s, now=None]



                                                                [A



t:   5%|▌         | 327/6023 [12:34<00:39, 144.76it/s, now=None]



chunk: 100%|██████████| 1/1 [06:13<00:00, 373.78s/it, now=None]

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


### Вырезка губ

In [1]:
import cv2

In [88]:
cap = cv2.VideoCapture('../video1.mp4')
mouth_cascade = cv2.CascadeClassifier('../../OpenCV/Cascades/Mouth.xml')

# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))
# print(frame_width, frame_height)

# out = cv2.VideoWriter('outpy_test.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 25, 
#                       (frame_width,frame_height))
while True:
    ret, img = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mouth = mouth_cascade.detectMultiScale(gray, 1.4, 20)
    
    croped_img = None
    for (x, y, w, h) in mouth:
        croped_img = img[y:y+h, x:x+w]
        
    print('Okey')
    if croped_img is None:
        continue
        
    cv2.imshow('test', croped_img)
#     out.write(croped_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey
Okey


KeyboardInterrupt: 

In [83]:
cv2.VideoWriter('asd.mp4', fourcc=cv2.VideoWriter_fourcc('M','J','P','G'), fps=25, frameSize=(frame_width,frame_height))

TypeError: VideoWriter() missing required argument 'apiPreference' (pos 2)

In [51]:
cap = cv2.VideoCapture('../video1.mp4')
mouth_cascade = cv2.CascadeClassifier('../../OpenCV/Cascades/Mouth.xml')

while True:
    ret, img = cap.read()
    if not ret:
        break
        
    cv2.imshow('img', img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mouth = mouth_cascade.detectMultiScale(gray, 1.4, 5)
    
    for (x, y, w, h) in mouth:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
cam = cv2.VideoCapture('../video1.mp4')
mouth_cascade = cv2.CascadeClassifier('../../OpenCV/Cascades/Mouth.xml')
face_cascade = cv2.CascadeClassifier('/home/alexey_kozlovsky/opencv/data/haarcascades/haarcascade_frontalface_default.xml')

settings = {
    'scaleFactor': 1.3, 
    'minNeighbors': 3, 
    'minSize': (50, 50), 
}

x_, y_, w_, h_ = None, None, None, None
while True:
    ret, img = cam.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, **settings)
    
    for (x, y, w, h) in faces:
        x_, y_, w_, h_ = x, y, w, h
        

while True:
    ret, img = cam.read()
   
    if not ret:
        break
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = face_cascade.detectMultiScale(img, 1.5, 5, flags=cv2.CASCADE_SCALE_IMAGE,
                                        minSize=(20, 20))
    
    for (x, y, w, h) in face:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cropped_img = gray[y:y+h, x:x+w]
        
        mouth = mouth_cascade.detectMultiScale(cropped_img, 1.2, 5)
        for (x1, y1, w1, h1) in mouth:
            cv2.rectangle(img, (x + x1, y + y1), (x + x1 + w1, y + y1 + h1), (0, 255, 0), 2)
#     for (x, y, w, h) in mouth:
#         cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
        
    
    cv2.imshow('cam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

In [9]:
settings = {
    'scaleFactor': 1.3, 
    'minNeighbors': 3, 
    'minSize': (50, 50), 
}

In [14]:
def rotate_img(img, ang):
    if ang == 0:
        return img
    height, width = img.shape[:2]
    rot_matrix = cv2.getRotationMatrix2D((width/2), (height/2), ang)
    result = cv2.warpAffine(img, rot_matrix, (width, height), flags=cv2.INTER_LINEAR)
    return result

In [12]:
def rotate_point(pos, img, ang):
    if ang == 0:
        return pos
    x = pos[0] - img.shape[1] * 0.4
    y = pos[1] - img.shape[0] * 0.4
    new_x = x * np.cos(np.radians(ang)) + y * np.sin(np.radians(ang)) + x
    new_y = -x * np.sin(np.radians(ang)) + y * np.cos(np.radians(ang)) + y
    return int(new_x), int(new_y), pos[2], pos[3]

In [49]:
cap = cv2.VideoCapture('../video1.mp4')
face = cv2.CascadeClassifier('/home/alexey_kozlovsky/opencv/data/haarcascades/haarcascade_frontalface_alt2.xml')

while True:
    ret, img = cap.read()
    if not ret:
        break
        
    for ang in [0, -25, 25]:
        rimg = rotate_img(img, ang)
        detected = face.detectMultiScale(rimg, **settings)
        if len(detected):
            detected = [rotate_point(detected[-1], img, -ang)]
            break
            
    for (x, y, w, h) in detected[-1:]:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
    cv2.imshow('facedetect', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

NameError: name 'rotate_img' is not defined

In [17]:
cv2.CASCADE_SCALE_IMAGE

AttributeError: module 'cv2.cv2' has no attribute 'CV_HAAR_SCALE_IMAGE'

In [3]:
cap = cv2.VideoCapture('../video1.mp4')
mouth_cascade = cv2.CascadeClassifier('../../OpenCV/Cascades/Mouth.xml')
face_cascade = cv2.CascadeClassifier('/home/alexey_kozlovsky/opencv/data/haarcascades/haarcascade_frontalface_default.xml')

settings = {
    'scaleFactor': 2, 
    'minNeighbors': 3, 
    'minSize': (50, 50), 
}

x_, y_, w_, h_ = None, None, None, None
while True:
    ret, img = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, **settings)
    
    for (x, y, w, h) in faces:
        x_, y_, w_, h_ = x, y, w, h
        break
    if len(faces):
        break
      
cap = cv2.VideoCapture('../video1.mp4')
x_current, y_current, w_current, h_current = 0, 0, 0, 0

while True:
    ret, img = cap.read()
    if not ret:
        break
        
    img = img[y_ + h_ // 2: y_ + h_, x_ : x_ + w_]
    mouths = mouth_cascade.detectMultiScale(img, 10, 5)
    
    
    for (x, y, w, h) in mouths:
        x_current, y_current, w_current, h_current = x, y, w, h
        
    cv2.rectangle(img, (x_current, y_current), 
                  (x_current + w_current, y_current + h_current), (255, 0, 0), 2)
    cv2.imshow('img', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
     
cap.release()
cv2.destroyAllWindows()

In [14]:
data = pd.read_csv('out1.csv')

In [16]:
data = data.loc[data['conf'] == 1]

,conf,end,start,word
3,1.0,1.200000,0.904665,привет
4,1.0,1.410000,1.200000,ещё
5,1.0,1.590000,1.410000,раз
6,1.0,2.430000,1.710000,расскажу
11,1.0,4.501097,3.960000,началось
...,...,...,...,...
466,1.0,227.610000,227.520000,с
467,1.0,228.554619,227.610000,алгоритмами
468,1.0,229.320000,228.600000,плюсами
469,1.0,229.500000,229.320000,и


In [18]:
word_dict = {}
for index, row in data.iterrows():
    word_dict[row['word']] = []
    
for index, row in data.iterrows():
    word_dict[row['word']].append((row['start'], row['end']))

In [24]:
cap = cv2.VideoCapture('../video1.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
millis = cap.get(cv2.CAP_PROP_FRAME_COUNT)
cap.release()

In [42]:
cap = cv2.VideoCapture('../video1.mp4')
cap.set(cv2.CAP_PROP_POS_MSEC, 1000)
print(cap.get(cv2.CAP_PROP_POS_FRAMES))

25.0


In [43]:
word_dict

{'привет': [(0.9046649999999999, 1.2)],
 'ещё': [(1.2, 1.41), (123.45, 123.75), (207.27, 207.57)],
 'раз': [(1.41, 1.59), (225.78, 226.02)],
 'расскажу': [(1.71, 2.43), (226.05, 226.5)],
 'началось': [(3.96, 4.501097)],
 'две': [(4.62, 5.01)],
 'идеи': [(5.01, 5.49)],
 'поделиться': [(6.75, 7.23)],
 'первая': [(7.62, 8.4), (14.67, 15.15)],
 'то': [(8.4, 8.64)],
 'что': [(8.64, 9.21),
  (25.32, 25.53),
  (62.52, 62.7),
  (100.98, 101.55),
  (113.97, 114.15),
  (218.97, 219.12)],
 'с': [(9.27, 9.45),
  (12.66, 12.72),
  (66.54, 66.63),
  (69.78, 69.87),
  (117.42, 117.84),
  (227.52, 227.61)],
 'этого': [(9.45, 9.72)],
 'года': [(9.72, 9.93)],
 'в': [(9.93, 9.99),
  (38.91, 38.940417),
  (74.7, 75.36),
  (110.46, 110.52),
  (129.87, 129.93),
  (150.0, 150.06),
  (166.53, 166.59)],
 'шаге': [(9.99, 10.32),
  (96.36, 96.9),
  (110.52, 111.05653600000001),
  (150.06, 150.36)],
 'есть': [(10.32, 10.62), (70.59, 70.8), (130.11, 130.29)],
 'четыре': [(10.62, 11.13)],
 'направления': [(11.13, 1

In [2]:
cam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('/home/alexey_kozlovsky/opencv/data/haarcascades/haarcascade_frontalface_default.xml')

while True:
    ret, img = cam.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.5, 10)
    
    for (x, y, w, h) in faces:
        cv2.imshow('face', img[y:y+h, x:x+w])
        
    cv2.imshow('img', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

In [4]:
test = {'word': ['sdf', 'sdfdsf'], 'start' : 12, 'end' : 19}

In [7]:
for i, word in enumerate(test['word']):
    print(i, word)

0 sdf
1 sdfdsf
